In [140]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


In [141]:
np.random.seed(100)

In [142]:
# There are going to be 
 
J = 2

# Across time period 

T = 3


# This might lead to bias see the assumption about J tending to infty and sthen T statoc
# This is the same sort of panel data bias!! Look a bit into microetrics


In [143]:
# All the parameter vector 
theta = np.array([1., 0.2, 0.7, 0.7, 0.2, 0.5, 0.1, 0.05])
beta_0 = 1.
beta_l = 0.2
beta_k = 0.7
beta = np.array([1, 0.2, 0.7])
# The coeff in the production function stuff 
alpha = 0.7
sigma_error_produciton = 0.2
simga_error_productivity = 0.5
simga_productivity = 0.1

# This is needed in the capital accumulation equation 
delta = 0.05

# The parameters for the shock in the labor function 
error_labour_optimization = np.random.normal(0, 0.05, (J, T))
# The random shock given to the labor (as in the estimated shock (idk in my mind this should be 0 
# but maybe i could ask Sabien I guess?))
eta_labor = np.random.normal(0, sigma_error_produciton, (J, T))
eta_production = np.random.normal(0, sigma_error_produciton, (J, T))

# The prices for the companies and the wage
p_eq = np.ones((J, T))
wage = np.ones((J, T))/2

# The capital value 
# The productivity shock has to be drawn from the stationart distribution of an ar(1) process and then it has to
# follow and Ar(1) process to begin with


In [144]:
def log_production(beta, l, k, omega, eta):
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    y = beta_0 + beta_l*l + beta_k*k + omega + eta 
    return y

In [145]:
def production(beta, labor, capital, omega, eta):
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    Y = (labor**beta_l)*(capital**beta_k)*np.exp(beta_0 + omega + eta)
    return Y 

In [146]:
def log_labor_optimal_error(beta, price, wage, omega, eta, k, iota): 
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    l = (1/(1-beta_l))*(np.log((price*beta_l)/wage)+ beta_0 + omega +iota + eta + beta_k*k)
    return l 

In [147]:
def log_labor_optimal(beta, price, wage, omega, eta, k): 
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    l = (1/(1-beta_l))*(np.log((price*beta_l)/wage)+ beta_0 + omega + eta + beta_k*k)
    return l 

In [148]:
def log_labor_optimal_meval(beta, price, wage, omega, eta, k): 
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    l = (1/(beta_l-1))*(np.log((wage)/(price*beta_l))- beta_0 - omega - eta - beta_k*k)
    return l 

In [149]:
def labor_optimal(beta, price, wage, omega, eta, capital):
    # Takes only non log values of production 
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    optimal_labor = (wage/(beta_l*capital**beta_k*np.exp(beta_0+omega+eta)*price))**(1/(beta_l-1))
    return optimal_labor

In [150]:

# This is not in log points and nothing takes log points until now
# You can log everything at the end if needed but not sure if you even have to at this point 
def investment(delta, omega, k):
    # There has to be some structure to the investment decision
    gamma = 0.1 
    # The gamma is set for all the time peruids going forward 
    inv = (delta + gamma*omega)*np.exp(k)
    return inv

In [151]:
def capital_formation(delta, capital, investment):
    capital_next= (1-delta)*capital + investment
    return capital_next

In [157]:
# Generating the productivity process for each of the generated firms
# The productivity shocks and their evolution
omega = np.zeros((J, T))
K = np.zeros((J, T))
inv = np.zeros((J, T))
L = np.zeros((J, T))

K_exp = np.zeros((J, T))
L_try = np.zeros((J, T))



Y = np.zeros((J, T))


# This is inconsistent because some of the formulas are in logs and some of the formulas are not in logs 

# There needs to be some sort of initialization for omega and capital 
omega[:,0] = np.random.normal(1, 0.05, J)
K[:, 0] = np.exp(np.random.normal(1, 0.5, J))
# in exponential form 
# K_exp[:, 0] = np.exp(K[:, 0])


inv[:, 0] = investment(delta, omega[:, 0], K[:, 0])

# Checking all the optimal labor stuff 
# L[:, 0] = log_labor_optimal(beta, p_eq[:,0], wage[:,0], omega[:, 0], eta_labor[:, 0], K[:,0])
L[:, 0] = labor_optimal(beta, p_eq[:,0], wage[:,0], omega[:, 0], eta_labor[:, 0], K[:,0])


# This is going to initiate all the production 
Y[:, 0] = production(beta, L[:, 0], K[:, 0], omega[:, 0], eta_production[:, 0])


for t in range(1, T):
    omega[:, t] = alpha * omega[:, t-1] + np.random.normal(0, 0.1, size=J)
    inv[:, t] = investment(delta, omega[:, t], K[:, t])
    K[:, t] = capital_formation(delta, K[:,t-1], inv[:,t-1])
    L[:, t] = labor_optimal(beta, p_eq[:,t], wage[:,t], omega[:, t], eta_labor[:, t], K[:,t])
    
    # Getting the optimal labor without the error term 
    Y[:, t] = production(beta, L[:, t], K[:, t], omega[:, t], eta_production[:, t])



print(L)
print(K)
print(Y)
print(omega)

print(np.log(L))
print(np.log(K))
print(np.log(Y))


[[17.32748627 30.65926178 27.45422672]
 [ 4.59627136  2.98430606  2.82238503]]
[[ 4.92739837 25.90190537 24.71587267]
 [ 1.1672979   1.58360227  1.610502  ]]
[[ 36.87854075 111.78793153  81.55308794]
 [ 10.69368752   7.35225668   7.94693925]]
[[1.03752224 0.59062566 0.35899405]
 [0.97720265 0.56079841 0.32574171]]
[[2.85229404 3.4229348  3.31252013]
 [1.5252454  1.09336724 1.03758228]]
[[1.59481113 3.25431653 3.20744566]
 [0.15469159 0.45970217 0.47654593]]
[[3.60762983 4.71660361 4.40125419]
 [2.36965362 1.9950073  2.07278685]]


In [153]:
# Now in each of these period 
seed = 100

df = pd.read_csv('/Users/popovici/Desktop/Thesis/markdown_estimation/data/market_100.csv').head(30)
df_grouped = df.groupby('time').head(2)
print(df_grouped)

       price      cost  product  time      Car1      Car2    mshare   profits  \
0   3.695253  1.415698        1     1  5.543405  5.891322  0.078804  0.179638   
1   3.756714  1.447384        2     1  5.278369  5.209202  0.107499  0.248250   
10  4.001494  1.413811        1     2  5.543405  5.891322  0.079771  0.206423   
11  4.140559  1.500000        2     2  5.278369  5.209202  0.105996  0.279890   
20  3.714313  1.434079        1     3  5.543405  5.891322  0.076635  0.174747   
21  3.753785  1.457664        2     3  5.278369  5.209202  0.105537  0.242325   

     markups  
0   1.610199  
1   1.595521  
10  1.830290  
11  1.760373  
20  1.590035  
21  1.575205  


In [154]:
eq_price = np.array(df_grouped['price'])
price_try = np.reshape(eq_price, (T, J)).T
print(price_try)

true_share = np.array(df_grouped['mshare'])
# print(true_share)

true_profit = np.array(df_grouped['profits'])
print(true_profit)


[[3.69525344 4.00149428 3.71431331]
 [3.75671434 4.14055935 3.75378478]]
[0.17963801 0.24825014 0.20642253 0.27988983 0.17474682 0.24232481]


In [155]:
# The idea is to optimize the new labor choice at the extracted prices from the market equiliborum: 

L_at_true_price =  log_labor_optimal(beta, p_eq[:,t], wage[:,t], omega[:, t], eta_labor[:, t], K[:,t])
